# PyTorch Assignment: Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Sonal Agrawal

### Convolutional Neural Network

Adapt the CNN example for MNIST digit classfication from Notebook 3A. 
Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> flatten ->
fully connected (256 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax 

Note: The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

In [67]:
### YOUR CODE HERE ###
import numpy as np
import torch
import torch.nn.functional as F

import torch.nn as nn

class MNIST_CNN_ASM(nn.Module):

        def __init__(self):

#             torch.Size([100, 1, 28, 28])
#             torch.Size([100, 32, 28, 28])
#             torch.Size([100, 32, 14, 14])
#             torch.Size([100, 64, 14, 14])
#             torch.Size([100, 64, 7, 7])
#             torch.Size([100, 64, 7, 7])            
            
            super().__init__()
            self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
            # convolutional layer (sees 16x16x16 tensor)
            self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
            # convolutional layer (sees 8x8x32 tensor)
            self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
            # max pooling layer
            self.pool = nn.MaxPool2d(2, 2)
            # linear layer (64 * 4 * 4 -> 500)
            self.fc1 = nn.Linear(64 * 3 * 3, 500)
            # linear layer (500 -> 10)
            self.fc2 = nn.Linear(500, 10)
            # dropout layer (p=0.25)
            self.dropout = nn.Dropout(0.25)

        def forward(self, x):
            #print(x.shape)
            x = self.pool(F.relu(self.conv1(x)))
            #print(x.shape)
            x = self.pool(F.relu(self.conv2(x)))
            #print(x.shape)
            x = self.pool(F.relu(self.conv3(x)))
            # flatten image input
            #print(x.shape)
            x = x.view(-1, 64 * 3 * 3)
            #print(x.shape)
            # add dropout layer
            x = self.dropout(x)
            # add 1st hidden layer, with relu activation function
            x = F.relu(self.fc1(x))
            #print(x.shape)
            # add dropout layer
            x = self.dropout(x)
            # add 2nd hidden layer, with relu activation function
            x = self.fc2(x)
            return x         



In [68]:
model = MNIST_CNN_ASM()
print(model)

MNIST_CNN_ASM(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=576, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [69]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

# Load the data
mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

## Training
# Instantiate model  
model = MNIST_CNN_ASM()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [71]:
import time
%time

# Iterate through train set minibatchs 
for epoch in trange(4):
    for images, labels in tqdm(train_loader):
        # Zero out the gradients
        optimizer.zero_grad()

        # Forward pass
        x = images.view(-1, 1, 28, 28)  # <---- change here 
        y = model(x)
        loss = criterion(y, labels)
        # Backward pass
        loss.backward()
        optimizer.step()


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.63 µs


In [72]:

## Testing
correct = 0
total = len(mnist_test)

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):
        # Forward pass
        x = images.view(-1, 1, 28, 28)  # <---- change here 
        y = model(x)

        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions == labels).float())

print('Test accuracy: {}'.format(correct/total))


Test accuracy: 0.9860000014305115


### Short answer

1\. How does the CNN compare in accuracy with yesterday's logistic regression and MLP models? How about training time?

`[Accuracy is better but time is also more]`

2\. How many trainable parameters are there in the CNN you built for this assignment?

*Note: The total of trainable parameters counts each element in a tensor. For example, a weight matrix that is 10x5 has 50 trainable parameters.*

`[576]`

3\. When would you use a CNN versus a logistic regression model or an MLP?

`[I'll go with CNN]`